In [20]:
from src.kr6 import KR6
import modern_robotics as mr
import numpy as np
from src.tools import*
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols

In [21]:
config1 = np.array([0,0,0,0,0,0])
config2 = np.array([0,-np.pi/2,np.pi/2,0,0,0])
config3 = np.array([0,0,np.pi/2,0,0,0])
config4 = np.array([0,0,0,0,0,0])
config5 = np.array([1,0.33,0.25,0,2,0])

kr6 = KR6(config1)

th1, th2, th3, th4, th5, th6 = sp.symbols("theta_1 theta_2 theta_3 theta_4 theta_5 theta_6")

# B_list with end effector in the wrist center
# B =sp.Matrix( [[-1    ,      0,    0 ,     0,     0,  0],
#                 [0    ,     -1,    -1,     0,    -1,  0],
#                 [0    ,      0,     0,    -1,     0, -1],
#                 [0    , -0.955,  -0.5,     0,     0,  0],
#                 [0.98 ,      0,     0,     0,     0,  0],
#                 [0    ,  0.035, 0.035,     0,     0, 0]])

B =sp.Matrix(  [[0    ,     0,     0,     1,     0,  1],
                [0    ,     1,     1,     0,     1,  0],
                [-1    ,    0,     0,     0,     0,  0],
                [0     ,    0,     0,     0,     0,  0],
                [-0.875,    0,     0,     0,     0,  0],
                [0     ,-0.875,  -0.42,   0,     0,  0]])


# L12=400
# L23=455
# L35=420
# L56=80
# B1=sp.Matrix([[0],[0],[-1],[0],[-L35-L23],[0]])
# B2=sp.Matrix([[0],[1],[0],[0],[0],[-L35-L23]])
# B3=sp.Matrix([[0],[1],[0],[0],[0],[-L35]])
# B4=sp.Matrix([[1],[0],[0],[0],[0],[0]])
# B5=sp.Matrix([[0],[1],[0],[0],[0],[0]])
# B6=sp.Matrix([[1],[0],[0],[0],[0],[0]])

Jb = sp.zeros(6,6)
Jb6= B[:,5]
Jb5= Ad(exp6(-B[:,5],th6)) * B[:,4]
Jb4= Ad(exp6(-B[:,5],th6)  * exp6(-B[:,4],th5)) * B[:,3]
Jb3= Ad(exp6(-B[:,5],th6)  * exp6(-B[:,4],th5) * exp6(-B[:,3],th4)) * B[:,2]
Jb2= Ad(exp6(-B[:,5],th6)  * exp6(-B[:,4],th5) * exp6(-B[:,3],th4) * exp6(-B[:,2],th3)) * B[:,1]
Jb1= Ad(exp6(-B[:,5],th6)  * exp6(-B[:,4],th5) * exp6(-B[:,3],th4) * exp6(-B[:,2],th3) * exp6(-B[:,1],th2)) * B[:,0]

Jb6.simplify()
Jb5.simplify()
Jb4.simplify()
Jb3.simplify()
Jb2.simplify()
Jb1.simplify()

Jb[:,0] = Jb1
Jb[:,1] = Jb2
Jb[:,2] = Jb3
Jb[:,3] = Jb4
Jb[:,4] = Jb5
Jb[:,5] = Jb6


Jb_11 = Jb[:3,:3]
Jb_12 = Jb[:3,3:]
Jb_21 = Jb[3:,:3]
Jb_22 = Jb[3:,3:]

c:\Users\Bendik\Documents\Git\TPK4170_project\src\tools.py:177: RuntimeWarning: invalid value encountered in sqrt
  s3_pos = np.sqrt(1-c3**2)
c:\Users\Bendik\Documents\Git\TPK4170_project\src\tools.py:178: RuntimeWarning: invalid value encountered in sqrt
  s3_neg = -np.sqrt(1-c3**2)


In [22]:
Jb_12_det = sp.det(Jb_12)
Jb_12_det

-sin(theta_5)*sin(theta_6)**2 - sin(theta_5)*cos(theta_6)**2

In [23]:
Jb_12_det_solve = sp.solve(Jb_12_det, th5, th6, dict=True)
Jb_12_det_solve

[{theta_5: 0}, {theta_5: pi}]

In [24]:
Jb_21_det = sp.det(Jb_21)

In [25]:
Jb_21_det_solve = sp.solve(Jb_21_det, th2, th3, th4, dict=True)
Jb_21_det_solve

[{theta_2: 3.14159265358979},
 {theta_2: -0.5*I*log(-(13.0*exp(I*theta_3) + 12.0)*exp(-I*theta_3)/(12.0*exp(I*theta_3) + 13.0))},
 {theta_2: -I*log(-sqrt(-(13.0*exp(I*theta_3) + 12.0)*exp(-I*theta_3)/(12.0*exp(I*theta_3) + 13.0)))},
 {theta_2: -I*log(-exp(-I*theta_3))},
 {theta_3: 0.0}]

In [50]:
# get a element from the Jb_21_det_solve dictionary
Jb_21_det_solve[1][th2]

-0.5*I*log(-(13.0*exp(I*theta_3) + 12.0)*exp(-I*theta_3)/(12.0*exp(I*theta_3) + 13.0))

In [51]:

a=Jb_21_det_solve[1][th2].subs(th3,np.pi/4).evalf()
b=Jb_21_det_solve[2][th2].subs(th3,2*np.pi/3).evalf()
c=Jb_21_det_solve[3][th2].subs(th3,2*np.pi/3).evalf()


1.19466427172745